In [1]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

# Carrega as variáveis de ambiente
load_dotenv()
s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

# Inicializa SparkSession com suporte a Iceberg + MinIO
spark = SparkSession.builder \
    .appName("IcebergBronzeToSilver") \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .config("spark.hadoop.fs.s3a.endpoint", s3_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Lista tabelas da camada bronze
bronze_tables = [row.tableName for row in spark.sql("SHOW TABLES IN local.bronze").collect()]

for table in bronze_tables:
    print(f"\n🔁 Processando tabela: {table}")

    # Lê a tabela da camada bronze
    df = spark.read.format("iceberg").load(f"local.bronze.{table}")

    # Exemplo de transformação: remove duplicados
    df_transformed = df.dropDuplicates()

    # Cria tabela silver (sobrescreve se necessário)
    #df_transformed.writeTo(f"local.silver.{table}").createOrReplace()
    df_transformed.writeTo(f"local.silver.{table}").overwritePartitions()
    print(f"✅ Tabela 'local.silver.{table}' criada com dados tratados.")

print("\n🚀 Todas as tabelas foram processadas para a camada silver.")


25/04/20 01:28:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/20 01:28:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/20 01:28:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.



🔁 Processando tabela: clientes


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `local`.`silver`.`clientes` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'OverwritePartitionsDynamic 'UnresolvedRelation [local, silver, clientes], [], false, true
+- Deduplicate [email#18, id#16, status#20, created_at#21, nome#17, data_cadastro#19]
   +- RelationV2[id#16, nome#17, email#18, data_cadastro#19, status#20, created_at#21] local.bronze.clientes local.bronze.clientes


In [2]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from datetime import datetime
import logging

# -------------------------
# Configuração do Logging
# -------------------------
def setup_logger():
    # Evita múltiplos handlers
    logger = logging.getLogger("minio_silver")
    if logger.handlers:  # Remove handlers existentes
        logger.handlers = []
    logger.setLevel(logging.INFO)
    handler = logging.StreamHandler()
    formatter = logging.Formatter(fmt='{"level": "%(levelname)s", "message": "%(message)s"}')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

logger = setup_logger()

# Carrega variáveis de ambiente do .env
load_dotenv()
s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

logger.info(f"🔐 Endpoint: {s3_endpoint} | Access Key: {s3_access_key[:4]}***")

# Cria sessão Spark com suporte a Iceberg + MinIO (S3A)
#spark = SparkSession.builder \
#    .appName("IcebergMinIOSilver") \
#    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
#    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
#    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
#    .config("spark.sql.catalog.local.type", "hadoop") \
#    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
#    .config("spark.hadoop.fs.s3a.endpoint", s3_endpoint) \
#    .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
#    .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
#    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
#    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
#    .config("spark.sql.catalog.local.default-namespace", "default") \
#    .getOrCreate()
# Cria sessão Spark com suporte a Iceberg + MinIO (S3A)
spark = SparkSession.builder \
    .appName("IcebergMinIOSilver") \
    .master("spark://spark-iceberg:7077")\
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .config("spark.hadoop.fs.s3a.endpoint", s3_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.sql.catalog.local.default-namespace", "default") \
    .config("spark.executor.memory", "2g")\
    .config("spark.executor.cores", "2")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.sql.shuffle.partitions", "8")\
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# Lista de tabelas bronze a processar (pode ser configurada dinamicamente ou via argumento)
bronze_tables = ["clientes"]  # Substitua por uma lista dinâmica se necessário

# Processa cada tabela bronze
for prefix in bronze_tables:
    logger.info(f"🔧 Processando prefixo para camada silver: {prefix}")

    # --------------------------------
    # Camada Silver
    # --------------------------------
    # Carrega os dados da camada bronze (apenas do dia atual para otimizar)
    try:
        silver_source_df = spark.table(f"local.bronze.{prefix}") \
            .filter(f"date(created_at) = '{datetime.now().strftime('%Y-%m-%d')}'")
    except Exception as e:
        logger.error(f"❌ Falha ao carregar tabela 'local.bronze.{prefix}': {str(e)}")
        continue

    # Validação: Remove duplicatas e nulos na chave primária (id)
    silver_source_df = silver_source_df.dropDuplicates(["id"]).filter("id IS NOT NULL")

    # Verifica se há dados a processar
    if silver_source_df.count() == 0:
        logger.info(f"ℹ️ Nenhum dado encontrado em 'local.bronze.{prefix}' para o dia atual. Pulando processamento.")
        continue

    # Exibe o schema da camada silver
    #logger.info("Schema da camada silver:")
    #silver_source_df.printSchema()

    # Prepara os campos para criação da tabela Iceberg
    cols = ", ".join([f"{field.name} STRING" for field in silver_source_df.schema.fields if field.name != "created_at"] + ["created_at TIMESTAMP"])

    # Cria tabela Iceberg na camada silver se não existir
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS local.silver.{prefix} (
            {cols}
        )
        USING iceberg
        PARTITIONED BY (days(created_at))
        TBLPROPERTIES (
            'write.format.default'='parquet',
            'write.parquet.compression-codec'='snappy',
            'write.target-file-size-bytes'='134217728',
            'commit.retry.num-retries'='10'
        )
    """)

    # Carrega a tabela silver existente
    silver_df = spark.table(f"local.silver.{prefix}")

    # Separa registros para atualização (existem na tabela silver) e inserção (novos)
    silver_existing_ids = silver_df.select("id").distinct()
    silver_update_df = silver_source_df.join(silver_existing_ids, "id", "inner")  # Registros que já existem
    silver_insert_df = silver_source_df.join(silver_existing_ids, "id", "left_anti")  # Registros novos

    # Escreve novos registros (inserção)
    if not silver_insert_df.isEmpty():
        silver_insert_df.writeTo(f"local.silver.{prefix}").append()
        logger.info(f"✅ Inseridos novos registros na tabela 'local.silver.{prefix}'.")

    # Escreve atualizações (sobrescreve partições afetadas)
    if not silver_update_df.isEmpty():
        silver_update_df.writeTo(f"local.silver.{prefix}").overwritePartitions()
        logger.info(f"✅ Atualizados registros existentes na tabela 'local.silver.{prefix}'.")

    # Manutenção na camada silver
    spark.sql(f"CALL local.system.rewrite_data_files(table => 'local.silver.{prefix}')")
    logger.info(f"🧹 Compactação de arquivos executada na tabela 'local.silver.{prefix}'.")

logger.info("🚀 Todas as tabelas silver foram processadas com sucesso.")

# Fecha a sessão Spark
spark.stop()

{"level": "INFO", "message": "🔐 Endpoint: http://minio:9000 | Access Key: mini***"}
{"level": "INFO", "message": "🔧 Processando prefixo para camada silver: clientes"}
{"level": "INFO", "message": "✅ Atualizados registros existentes na tabela 'local.silver.clientes'."}
{"level": "INFO", "message": "🧹 Compactação de arquivos executada na tabela 'local.silver.clientes'."}
{"level": "INFO", "message": "🚀 Todas as tabelas silver foram processadas com sucesso."}
